# Scraping for bad stats
bad stats are technical fouls, ejections and flagrant fouls

In [1]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import pickle

###### parses html of any url provided

In [2]:
def getAndParseUrl(url):
    results = requests.get(url)
    time.sleep(1)
    soup = BeautifulSoup(results.text, 'html.parser')
    return soup

###### scrape foxsports.com to get the bad stats

In [3]:
def get_bad_stats():
    
    # container for each player's bad stats
    bad = []
    
    # loop through each page of the url (12 pages total)
    for i in range(1,13):
        soup = getAndParseUrl(f'https://www.foxsports.com/nba/stats?season=2018&category=MISC&group=1&sort=8&time=0&pos=0&team=0&qual=1&sortOrder=0&opp=0&page={i}')
        
        # each row is a player
        rows = soup.select('table.wisbb_standardTable tbody tr')
        labels = [t.text.strip('\n') for t in soup.select('table.wisbb_standardTable thead tr th')]
        for row in tqdm(rows):
            playa = {}
            stats = row.select('td')
            playa['Name'] = stats[0].text.replace('\n','')
            for j in range(-5,-2):
                playa[labels[j]] = stats[j].text
            bad.append(playa)
    return bad

In [4]:
bad_stats = get_bad_stats()

100%|██████████| 27/27 [00:00<00:00, 1986.46it/s]


In [5]:
with open('bad_stats.pickle', 'wb') as outfile:
    pickle.dump(bad_stats, outfile)

outfile.close()

In [6]:
bad_stats

[{'Name': '1Westbrook, RussellWestbrook, R.  HOU',
  'FLAG': '4',
  'TECH': '16',
  'EJECT': '0'},
 {'Name': '2Pachulia, ZazaPachulia, Z. ',
  'FLAG': '3',
  'TECH': '4',
  'EJECT': '1'},
 {'Name': '2Len, AlexLen, A.  ATL', 'FLAG': '3', 'TECH': '0', 'EJECT': '0'},
 {'Name': '2Embiid, JoelEmbiid, J.  PHI',
  'FLAG': '3',
  'TECH': '10',
  'EJECT': '0'},
 {'Name': '5Chandler, TysonChandler, T.  HOU',
  'FLAG': '2',
  'TECH': '3',
  'EJECT': '0'},
 {'Name': '5Williams, MarvinWilliams, M.  CHA',
  'FLAG': '2',
  'TECH': '2',
  'EJECT': '0'},
 {'Name': '5Young, ThaddeusYoung, T.  CHI',
  'FLAG': '2',
  'TECH': '4',
  'EJECT': '1'},
 {'Name': '5Cousins, DeMarcusCousins, D.  LAL',
  'FLAG': '2',
  'TECH': '7',
  'EJECT': '1'},
 {'Name': '5Favors, DerrickFavors, D.  NOP',
  'FLAG': '2',
  'TECH': '3',
  'EJECT': '1'},
 {'Name': '5Morris, MarcusMorris, M.  NYK',
  'FLAG': '2',
  'TECH': '9',
  'EJECT': '1'},
 {'Name': '5Scott, MikeScott, M.  PHI',
  'FLAG': '2',
  'TECH': '3',
  'EJECT': '0'},


In [8]:
sp = getAndParseUrl('https://basketball.fantasysports.yahoo.com/nba/express_settings?type=point')

In [52]:
yahoo_scoring = [[t.text for t in el.select('td')] for el in  sp.select('table.settings tr')][-11:]

In [54]:
for el in yahoo_scoring:
    el[1] = el[1][:-7]

In [56]:
for el in yahoo_scoring:
    el[1] = float(el[1])

In [57]:
yahoo_scoring

[['Field Goals Attempted', -0.45],
 ['Field Goals Made', 1.0],
 ['Free Throws Attempted', -0.75],
 ['Free Throws Made', 1.0],
 ['3-point Shots Made', 3.0],
 ['Points Scored', 0.5],
 ['Total Rebounds', 1.5],
 ['Assists', 2.0],
 ['Steals', 3.0],
 ['Blocked Shots', 3.0],
 ['Turnovers', -2.0]]

In [59]:
with open('yahoo_scoring.pickle', 'wb') as outfile:
    pickle.dump(yahoo_scoring, outfile)

outfile.close()